In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv

import KY_utils.KY_interpretable_functions as interpret
import KY_utils.KY_RiskSLIM as slim
import KY_utils.KY_stumps as stumps
import KY_utils.KY_Corel as Corel

# restore saved variables
%store -r summary_property6_ky_inter_model

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\kentucky\ky_fl_combined\KY Model


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### EBM & CART

In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test/kentucky/models/ky_fl_combined/data preparation/KY_cleaned")
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test/kentucky/models/ky_fl_combined/data preparation/FL_cleaned")

## split x and y 
KY_X = KY_data.loc[:,:'current_violence']
KY_Y = KY_data['recid_property_six_month'].values
FL_X = FL_data.loc[:,:'current_violence']
FL_Y = FL_data['recid_property_six_month'].values

In [3]:
#### CART
depth = [4,5]
cart_summary = advance.CART(KY_X, KY_Y, FL_X, FL_Y, depth, seed=816)

### EBM
estimators = [20]
depth = [2]
learning_rate = [0.5]
ebm_summary = advance.EBM(KY_X, KY_Y, FL_X, FL_Y, learning_rate, depth, estimators, seed=816)

In [4]:
cart_summary,  ebm_summary

({'best_auc': 0.8177417291332514,
  'auc_diff': 0.011805015643108319,
  'best_param': {'max_depth': 4},
  'FL_score': 0.6258411769880365},
 {'best_auc': 0.8326026655722354,
  'auc_diff': 0.024666357228204205,
  'best_param': {'learning_rate': 0.5,
   'max_tree_splits': 2,
   'n_estimators': 20},
  'FL_score': 0.5040711866643208})

### Lasso Stumps

In [7]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test/kentucky/models/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test/kentucky/models/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence>=1']
KY_Y = KY_stumps['recid_property_six_month'].values
FL_X = FL_stumps.loc[:,:'current_violence>=1']
FL_Y = FL_stumps['recid_property_six_month'].values

## columns 
cols = KY_X.columns

In [8]:
c_grid = {'C':[0.03, 0.05, 0.07]}
stumps_summary = stumps.stump_cv(KY_X, KY_Y, FL_X, FL_Y, cols, c_grid, seed=816)

In [9]:
stumps_summary

{'best_auc': 0.8385703643892033,
 'best_params': {'C': 0.03},
 'auc_diffs': 0.026543810657584377,
 'FL_score': 0.6626935256861366}

### RiskSLIM

In [14]:
## train on best param chosen by Lasso Stumps from above
lasso = LogisticRegression(class_weight='balanced', solver='liblinear', penalty='l1', C=0.03, random_state=816).fit(KY_X, KY_Y)
selected_features = cols[lasso.coef_[0] != 0].tolist()
len(selected_features)

67

In [15]:
### Subset features
sub_FL_X = FL_stumps.loc[:, selected_features]
sub_KY_X = KY_stumps.loc[:, selected_features]
sub_KY_X.insert(0, '(Intercept)', 1)

In [16]:
riskslim_summary = slim.risk_cv(sub_KY_X, KY_Y, sub_FL_X, FL_Y, 
                                y_label = 'general2', 
                                max_coef = 20, 
                                max_coef_number = 10, 
                                max_runtime=400, 
                                c=1e-3, 
                                seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
09/17/19 @ 09:31 PM | 2781 rows in lookup table
09/17/19 @ 09:31 PM | ------------------------------------------------------------
09/17/19 @ 09:31 PM | runnning initialization procedure
09/17/19 @ 09:31 PM | ------------------------------------------------------------
09/17/19 @ 09:31 PM | CPA produced 2 cuts
09/17/19 @ 09:31 PM | running naive rounding on 164 solutions
09/17/19 @ 09:31 PM | best objective value: 0.0319
09/17/19 @ 09:31 PM | rounding produced 5 integer solutions
09/17/19 @ 09:31 PM | best objective value is 0.0354
09/17/19 @ 09:31 PM | running sequential rounding on 164 solutions
09/17/19 @ 09:31 PM | best objective value: 0.0319
09/17/19 @ 09:31 PM | sequential rounding produced 3 integer solutions
09/17/19 @ 09:31 PM | best objective value: 0.0352
09/17/19 @ 09:31 PM | polishing 8 solutions
09/17/19 @ 09:31 PM | best objective value: 0.0352
09/17/19 @ 09:31 PM | polishing produced 3 integer solutions
09/17/1

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
09/17/19 @ 09:32 PM | adding 258 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.0341.
Tried aggregator 1 time.
Reduced MIP has 69 rows, 138 columns, and 271 nonzeros.
Reduced MIP has 67 binaries, 69 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.16 ticks)
Probing time = 0.00 sec. (0.07 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.09 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Varia

In [17]:
riskslim_summary

{'FL_Score': 0.6395254222378607}

### Corel

In [2]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test/kentucky/models/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test/kentucky/models/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence>=1']
KY_Y = KY_stumps['recid_property_six_month'].values
FL_X = FL_stumps.loc[:,:'current_violence>=1']
FL_Y = FL_stumps['recid_property_six_month'].values

In [3]:
KY_score, FL_score = Corel.corel_cv(KY_X, KY_Y, FL_X, FL_Y, max_card=2, c=1e-3, seed=816)

In [17]:
#### save results
summary_property6_ky_inter_model = {"CART": cart_summary,
                                   "EBM": ebm_summary,
                                   "Stumps": stumps_summary,
                                   "RiskSLIM": riskslim_summary, 
                                   "Corel": FL_score}
%store summary_property6_ky_inter_model

Stored 'summary_general2_ky_inter_model' (dict)


In [20]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\KY_FL\\KY Model\\interpretable\\Six Month\\"
results = [["Property",             
            round(cart_summary['FL_score'],3), 
            round(ebm_summary['FL_score'], 3), 
            round(stumps_summary['FL_score'],3), 
            round(riskslim_summary['FL_Score'], 3), 
            round(FL_score, 3)]]
with open(path + 'Six Month KY_inter_model.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)